In [1]:
pip install PyPDF2 qrcode[pil] pillow

  Using cached qrcode-7.4.2-py3-none-any.whl.metadata (17 kB)
  Using cached pypng-0.20220715.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   --- ----------------------------------- 20.5/232.6 kB 165.2 kB/s eta 0:00:02
   --- ----------------------------------- 20.5/232.6 kB 165.2 kB/s eta 0:00:02
   ----- --------------------------------- 30.7/232.6 kB 187.9 kB/s eta 0:00:02
   ----- --------------------------------- 30.7/232.6 kB 187.9 kB/s eta 0:00:02
   -------- ------------------------------ 51.2/232.6 kB 201.8 kB/s eta 0:00:01
   ------------- ------------------------- 81.9/232.6 kB 255.1 kB/s eta 0:00:01
   ------------- ------------------------- 81.9/232.6 kB 255.1 kB/s eta 0:00:01
   ---------------- --------------------- 102.4/232.6 kB 268.7 kB/s eta 0:00:01
   --------------------- ---------------- 133.1/232.6 kB 302.7 kB/s eta 0:00:01
   ----

In [2]:
import os
import tempfile
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
import qrcode
from PIL import Image, ImageTk
import zipfile

class MP3toQRCodeApp:
    def __init__(self, root):
        self.root = root
        self.root.title("MP3 QR Generator")
        self.qrcodes = []  # Initialize qrcodes attribute
        self.create_widgets()
        
    def create_widgets(self):
        # Set the color theme
        self.root.configure(bg="#f0f0f0")
        
        # File upload section
        self.upload_button = tk.Button(self.root, text="Upload MP3", command=self.upload_file, bg="#4CAF50", fg="white")
        self.upload_button.pack(pady=10)

        # QR code preview section
        self.preview_label = tk.Label(self.root, text="QR Code Preview:", bg="#f0f0f0")
        self.preview_label.pack()
        self.preview_canvas = tk.Canvas(self.root, width=300, height=300, bg="white")
        self.preview_canvas.pack(pady=10)

        # Download button
        self.download_button = tk.Button(self.root, text="Download QR Code(s)", command=self.download_qrcodes, bg="#2196F3", fg="white")
        self.download_button.pack(pady=10)

    def upload_file(self):
        self.filepath = filedialog.askopenfilename(filetypes=[("MP3 files", "*.mp3")])
        if self.filepath:
            threading.Thread(target=self.generate_qrcodes).start()

    def generate_qrcodes(self):
        try:
            # Generate QR code with MP3 file path or URL
            self.qrcodes = []

            # Here we just use the file path as data. In a real-world scenario, you'd use a URL.
            mp3_url = f'file:///{self.filepath.replace(" ", "%20")}'

            # Generate QR code
            qr = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_M,
                box_size=10,
                border=4,
            )
            qr.add_data(mp3_url)
            qr.make(fit=True)

            img = qr.make_image(fill='black', back_color='white')
            self.qrcodes.append(img)

            self.show_preview()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def show_preview(self):
        if self.qrcodes:
            qr_img = self.qrcodes[0].resize((300, 300), Image.LANCZOS)
            self.qr_photo = ImageTk.PhotoImage(qr_img)
            self.preview_canvas.create_image(0, 0, anchor=tk.NW, image=self.qr_photo)

    def download_qrcodes(self):
        if not hasattr(self, 'qrcodes') or not self.qrcodes:
            messagebox.showwarning("No QR Codes", "No QR codes generated to download.")
            return

        threading.Thread(target=self._download_qrcodes_thread).start()

    def _download_qrcodes_thread(self):
        save_path = filedialog.askdirectory()
        if not save_path:
            return

        try:
            temp_dir = tempfile.mkdtemp()
            zip_path = os.path.join(temp_dir, "qrcodes.zip")
            with zipfile.ZipFile(zip_path, 'w') as zipf:
                for i, qr_img in enumerate(self.qrcodes):
                    img_path = os.path.join(temp_dir, f'qr_code.png')
                    qr_img.save(img_path)
                    zipf.write(img_path, f'qr_code.png')

            final_zip_path = os.path.join(save_path, "qrcodes.zip")
            os.rename(zip_path, final_zip_path)
            messagebox.showinfo("Download Complete", f"QR Codes downloaded successfully to {final_zip_path}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

if __name__ == "__main__":
    root = tk.Tk()
    app = MP3toQRCodeApp(root)
    root.mainloop()
